In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix,classification_report

In [ ]:
train=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
test=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")

In [ ]:
train.shape,test.shape

In [ ]:
train.info()

In [ ]:
train.Made_Purchase.value_counts().plot(kind='bar')

In [ ]:
X_trainData=train.drop('Made_Purchase',axis=1)
Y=train['Made_Purchase'].astype('int')

In [ ]:
X_trainData.shape,test.shape

In [ ]:
X_trainData.describe()

In [ ]:
X_trainData.isnull().sum()*100/len(X_trainData)

In [ ]:
numeric_cols=X_trainData.select_dtypes(include=np.number).columns.tolist()
categorical_cols=X_trainData.select_dtypes('object').columns.tolist()

In [ ]:
numeric_cols

In [ ]:
categorical_cols

In [ ]:
from sklearn.impute import KNNImputer
imputer=KNNImputer(n_neighbors=10, missing_values=np.nan)
imputer.fit(X_trainData[numeric_cols])
X_trainData[numeric_cols]=imputer.transform(X_trainData[numeric_cols])
test[numeric_cols]=imputer.transform(test[numeric_cols])

In [ ]:
sns.distplot(X_trainData['ProductDescriptionPage_Duration'])

In [ ]:
sns.distplot(X_trainData['HomePage'])

In [ ]:
X_trainData[categorical_cols].isnull().sum()

In [ ]:
imputer_cg=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer_cg.fit(X_trainData[categorical_cols])
X_trainData[categorical_cols]=imputer_cg.transform(X_trainData[categorical_cols])
test[categorical_cols]=imputer_cg.transform(test[categorical_cols])

In [ ]:
target=pd.Series(Y) 
data =  pd.concat([X_trainData,target],axis=1)


In [ ]:
# Co-relation matrix
corr=data.corr()
f,ax=plt.subplots(figsize=(21,11))
cmap=sns.diverging_palette(220,10,as_cmap=True)
heatmap=sns.heatmap(corr,cmap=cmap,center=0.0,vmax=1,linewidths=1,ax=ax,annot=True)
plt.show()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first')
ohe.fit(X_trainData[categorical_cols])
train_data=ohe.transform(X_trainData[categorical_cols]).toarray()
test_data=ohe.transform(test[categorical_cols]).toarray()

In [ ]:
feature_names = ohe.get_feature_names(categorical_cols)
train_df = pd.concat([X_trainData[numeric_cols], 
               pd.DataFrame(data=train_data,columns=feature_names).astype(int)], axis=1)
test_df = pd.concat([test[numeric_cols], 
               pd.DataFrame(data=test_data,columns=feature_names).astype(int)], axis=1)

In [ ]:
from sklearn.preprocessing import PowerTransformer
scaler=PowerTransformer()
scaler.fit(train_df[numeric_cols])
train_df[numeric_cols]=scaler.transform(train_df[numeric_cols])
test_df[numeric_cols]=scaler.transform(test_df[numeric_cols])

In [ ]:
sns.scatterplot(data=data,y='GoogleMetric:Page Values',
                      x='Made_Purchase',
                      hue='Made_Purchase')

In [ ]:
sns.scatterplot(data=data,y='ProductDescriptionPage_Duration',
                      x='Made_Purchase',
                      hue='Made_Purchase')

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(train_df[numeric_cols])

# plot the cumulative explained variance ratio
import matplotlib.pyplot as plt
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.show()

n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.99) + 1
pca = PCA(n_components=n_components)



In [ ]:
# fit and transform the selected columns of train_df and test_df
train_df_pca = pd.DataFrame(pca.fit_transform(train_df[numeric_cols]), columns=['PCA_' + str(i) for i in range(1, n_components+1)])
test_df_pca = pd.DataFrame(pca.transform(test_df[numeric_cols]), columns=['PCA_' + str(i) for i in range(1, n_components+1)])

# add the PCA columns to train_df and test_df
train_df = pd.concat([train_df, train_df_pca], axis=1)
test_df = pd.concat([test_df, test_df_pca], axis=1)

# drop the original numeric columns
train_df.drop(numeric_cols, axis=1, inplace=True)
test_df.drop(numeric_cols, axis=1, inplace=True)

In [ ]:
X=train_df
test=test_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=le.fit_transform(Y)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

# shuffle the training data
X,Y = shuffle(X,Y, random_state=0)

# create an LDA object
model = LDA()

# fit the LDA model on the training data
model.fit(X,Y)

# perform 5-fold cross-validation on the training data
scores = cross_val_score(model,X,Y, cv=5)

# print the mean cross-validation score
print('Cross-validation scores:', scores)
print('Mean cross-validation score:', scores.mean())

In [ ]:
from sklearn.model_selection import train_test_split
def classify(model):
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)
    model.fit(x_train,y_train)
    test_pred=model.predict(x_test) 
    print('Accuracy:',accuracy_score(y_test,test_pred))
    print('Mean Squared Error',mean_squared_error(y_test,test_pred))
    print(confusion_matrix(y_test,test_pred))
    print(classification_report(y_test,test_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='liblinear',penalty='l2', random_state=42)
classify(model)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(criterion='gini', 
                             splitter='best', 
                             max_depth=5, 
                             min_samples_split=2, 
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features=None, 
                             random_state=42, 
                             max_leaf_nodes=None, 
                             min_impurity_decrease=0.0)
classify(model)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=200,
                             criterion='gini',
                             max_depth=5, 
                             min_samples_split=3, 
                             min_samples_leaf=6, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', 
                             max_leaf_nodes=None, 
                             min_impurity_decrease=0.0,
                             random_state=42)
classify(model)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=30,weights="uniform",metric="euclidean")
classify(knn)

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier(max_depth = 3,
                              gamma = 0.5,
                              n_estimators = 200,
                              learning_rate = 0.1,
                              min_child_weight = 7,
                              subsample = 0.8,
                              reg_lambda = 10,
                              reg_alpha = 15,
                              colsample_bytree = 0.7,
                              colsample_bylevel = 0.7,
                              colsample_bynode = 0.7,
                              random_state = 42,
                              objective="binary:logistic")
classify(xgb)

In [ ]:
from sklearn.ensemble import VotingClassifier
clf1=KNeighborsClassifier(n_neighbors=50)
clf3=LogisticRegression(solver='liblinear',penalty='l2', random_state=0)
eclf1 = VotingClassifier(estimators=[('knn', clf1),('lg',clf3)], voting='soft')
classify(eclf1)

In [ ]:
clf1=KNeighborsClassifier(n_neighbors=50)
clf2=LogisticRegression(solver='liblinear',penalty='l2', random_state=0)
model = VotingClassifier(estimators=[('knn', clf1),('lg',clf2)], voting='soft')
model.fit(X,Y)

In [ ]:
pred=model.predict(test).astype(dtype=bool)
pred

In [ ]:
sample=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv")
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])

In [ ]:
submission['id']=[i for i in range(len(pred))]
submission["Made_Purchase"]=pred

In [ ]:
submission.to_csv('submission.csv',index=False)